# $\psi_4$

### Author: Zach Etienne



<a id='initializenrpy'></a>

# Step 1: Initialize core NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1.a: import all needed modules from NRPy+:
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm

# Step 1.b: Set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Spherical")

# Step 1.c: Given the chosen coordinate system, set up 
#           corresponding reference metric and needed
#           reference metric quantities
# The following function call sets up the reference metric
#    and related quantities, including rescaling matrices ReDD,
#    ReU, and hatted quantities.
rfm.reference_metric()

# Step 1.d: Set spatial dimension (must be 3 for BSSN, as BSSN is 
#           a 3+1-dimensional decomposition of the general 
#           relativistic field equations)
DIM = 3

# Step 1.e: Import all ADM quantities as written in terms of BSSN quantities
import BSSN.ADM_in_terms_of_BSSN as AB
AB.ADM_in_terms_of_BSSN()

initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX


# Step 2: Constructing the 3-Riemann tensor $R_{ik\ell m}$
Analogously to Christoffel symbols, the Riemann tensor is a measure of the curvature of an $N$-dimensional manifold. Thus the 3-Riemann tensor is not simply a projection of the 4-Riemann tensor (see e.g., Eq. 2.7 of [Campanelli *et al* (1998)](https://arxiv.org/pdf/gr-qc/9803058.pdf) for the relation between 4-Riemann and 3-Riemann), as $N$-dimensional Riemann tensors are meant to define a notion of curvature given only the associated $N$-dimensional metric. 

So, given the ADM 3-metric, the Riemann tensor in arbitrary dimension is given by the 3-dimensional version of Eq. 1.19 in Baumgarte & Shapiro's *Numerical Relativity*. I.e.,

$$
R^i_{jkl} = \partial_k \Gamma^{i}_{jl} - \partial_l \Gamma^{i}_{jk} + \Gamma^i_{mk} \Gamma^m_{jl} - \Gamma^{i}_{ml} \Gamma^{m}_{jk},
$$
where $\Gamma^i_{jk}$ is the Christoffel symbol associated with the 3-metric $\gamma_{ij}$:

$$
\Gamma^l_{ij} = \frac{1}{2} \gamma^{lk} \left(\gamma_{ki,j} + \gamma_{kj,i} - \gamma_{ij,k} \right) 
$$

Notice that this equation for the Riemann tensor is equivalent to the equation given in the Wikipedia article on [Formulas in Riemannian geometry](https://en.wikipedia.org/w/index.php?title=List_of_formulas_in_Riemannian_geometry&oldid=882667524):

$$
R^\ell{}_{ijk}=
\partial_j \Gamma^\ell{}_{ik}-\partial_k\Gamma^\ell{}_{ij}
+\Gamma^\ell{}_{js}\Gamma_{ik}^s-\Gamma^\ell{}_{ks}\Gamma^s{}_{ij},
$$
with the replacements $i\to \ell$, $j\to i$, $k\to j$, $l\to k$, and $s\to m$. Wikipedia also provides a simpler form in terms of second-derivatives of three-metric itself (using the definition of Christoffel symbol), so that we need not define derivatives of the Christoffel symbol:

$$
R_{ik\ell m}=\frac{1}{2}\left(
\gamma_{im,k\ell} 
+ \gamma_{k\ell,im}
- \gamma_{i\ell,km}
- \gamma_{km,i\ell} \right)
+\gamma_{np} \left(
\Gamma^n{}_{k\ell} \Gamma^p{}_{im} - 
\Gamma^n{}_{km} \Gamma^p{}_{i\ell} \right).
$$

First we construct the term on the left:

In [2]:
# Step 2: Construct the (rank-4) Riemann curvature tensor associated with the ADM 3-metric:
RDDDD = ixp.zerorank4()
gammaDDdDD = AB.gammaDDdDD

for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                RDDDD[i][k][l][m] = sp.Rational(1,2) * \
                    (gammaDDdDD[i][m][k][l] + gammaDDdDD[k][l][i][m] - gammaDDdDD[i][l][k][m] - gammaDDdDD[k][m][i][l])

... then we add the term on the right:

In [3]:
gammaDD = AB.gammaDD
GammaUDD = AB.GammaUDD

for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                for n in range(DIM):
                    for p in range(DIM):
                        RDDDD[i][k][l][m] += gammaDD[n][p] * \
                            (GammaUDD[n][k][l]*GammaUDD[p][i][m] - GammaUDD[n][k][m]*GammaUDD[p][i][l])

# Step 3: Constructing the rank-4 tensor in Term 1 of $\psi_4$: $R_{ijkl} + 2 K_{i[k} K_{l]j} $

Following Eq. 5.1 in [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf), the rank-4 tensor in the first term of $\psi_4$ is given by

$$
R_{ijkl} + 2 K_{i[k} K_{l]j} = R_{ijkl} + K_{ik} K_{lj} - K_{il} K_{kj}
$$

In [4]:
# Step 3: Construct the (rank-4) tensor in term 1 of psi_4 (referring to Eq 5.1 in 
#   Baker, Campanelli, Lousto (2001); https://arxiv.org/pdf/gr-qc/0104063.pdf
rank4term1 = ixp.zerorank4()
KDD = AB.KDD

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                rank4term1[i][j][k][l] = RDDDD[i][j][k][l] + KDD[i][k]*KDD[l][j] - KDD[i][l]*KDD[k][j]

# Step 4: Constructing the rank-3 tensor in Term 2 of $\psi_4$: 

Following Eq. 5.1 in [Baker, Campanelli, Lousto (2001)[https://arxiv.org/pdf/gr-qc/0104063.pdf], the rank-3 tensor in the second term of $\psi_4$ is given by

$$
-8 \left(K_{j[k,l]} + \Gamma^{p}_{j[k} K_{l]p} \right) = 
$$
First let's construct the first term in this sum: $K_{j[k,l]} = \frac{1}{2} (K_{jk,l} - K_{jl,k})$:

In [5]:
# Step 4: Construct the (rank-3) tensor in term 2 of psi_4 (referring to Eq 5.1 in 
#   Baker, Campanelli, Lousto (2001); https://arxiv.org/pdf/gr-qc/0104063.pdf
rank3term2 = ixp.zerorank3()
KDDdD = AB.KDDdD

for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            rank3term2[j][k][l] = sp.Rational(1,2)*(KDDdD[j][k][l] - KDDdD[j][l][k])

... then we construct the secon term in this sum: $\Gamma^{p}_{j[k} K_{l]p} = \frac{1}{2} (\Gamma^{p}_{jk} K_{lp}-\Gamma^{p}_{jl} K_{kp})$:

In [6]:
for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for p in range(DIM):
                rank3term2[j][k][l] += sp.Rational(1,2)*(GammaUDD[p][j][k]*KDD[l][p] - GammaUDD[p][j][l]*KDD[k][p])

Finally, we multiply the term by $-8$:

In [7]:
for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            rank3term2[j][k][l] *= sp.sympify(-8)

# Step 5: Constructing the rank-2 tensor in term 3 of $\psi_4$:

Following Eq. 5.1 in [Baker, Campanelli, Lousto (2001)[https://arxiv.org/pdf/gr-qc/0104063.pdf], the rank-2 tensor in the third term of $\psi_4$ is given by

$$
+4 \left(R_{jl} - K_{jp} K^p_l + K K_{jl} \right),
$$
where
\begin{align}
R_{jl} &= R^i_{jil} \\
&= \gamma^{im} R_{ijml} \\
K &= K^i_i \\
&= \gamma^{im} K_{im}
\end{align}

First let's build the first term in parentheses

In [8]:
# Step 5: Construct the (rank-2) tensor in term 3 of psi_4 (referring to Eq 5.1 in 
#   Baker, Campanelli, Lousto (2001); https://arxiv.org/pdf/gr-qc/0104063.pdf
rank2term3 = ixp.zerorank2()
gammaUU = AB.gammaUU

for j in range(DIM):
    for l in range(DIM):
        for i in range(DIM):
            for m in range(DIM):
                rank2term3[j][l] += gammaUU[i][m]*RDDDD[i][j][m][l]

... then we add on the second term in parentheses, where $K^p_l = \gamma^{mp} K_{ml}$

In [9]:
for j in range(DIM):
    for l in range(DIM):
        for m in range(DIM):
            for p in range(DIM):
                rank2term3[j][l] += - KDD[j][p]*gammaUU[p][m]*KDD[m][l]

Finally we add the third term in parentheses, and multiply all terms by $+4$:

In [10]:
for j in range(DIM):
    for l in range(DIM):
        for i in range(DIM):
            for m in range(DIM):
                rank2term3[j][l] += gammaUU[i][m]*KDD[i][m]*KDD[j][l]
for j in range(DIM):
    for l in range(DIM):
        rank2term3[j][l] *= sp.sympify(4)

# Step 6: Constructing $\psi_4$ through contractions of the above terms with an arbitrary tetrad

